# Verifying ISS for pendulum

In [ ]:
from dreal import *
import torch 
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import timeit

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
Wxh = torch.tensor([[-0.1140,  0.0439, -0.3238,  0.2700,  0.0452, -0.4382,  0.2214, -0.2433,
         -0.1157, -0.2126,  0.0531,  0.1103, -0.1372,  0.1931, -0.1770,  0.2604,
          0.3106, -0.1300, -0.2426,  0.6030,  0.2640,  0.0964,  0.2053, -0.2922,
          0.1671, -0.2622, -0.2435,  0.2114,  0.2597,  0.2373, -0.3134,  2.1553,
          0.2730, -0.0258, -0.2683, -0.2195, -0.1279,  0.2932,  0.1970, -0.1146,
          0.1975,  0.1522, -0.1705,  0.1871,  0.2056, -0.0577,  0.2288,  0.0815,
          0.0461, -0.1441]])

Wyh = torch.tensor([[ 0.0339, -0.3451, -0.1212,  0.0220, -0.2942, -0.1812, -0.1005,  0.2557,
          0.1621, -0.0168,  0.1531, -0.0040,  0.0866,  0.0298, -0.0702, -0.0009,
         -0.4154, -0.2520,  0.0855,  0.4797,  0.1072,  0.2040,  0.2764, -0.0112,
          0.1596,  0.0597,  0.0607, -0.0659, -0.1365,  0.1199,  0.0903, -0.6860,
          0.2007,  0.0089, -0.2271,  0.0666, -0.0322,  0.1811,  0.1034,  0.3887,
         -0.1970,  0.0553,  0.0251,  0.0012,  0.1087, -0.1363,  0.0800,  0.0625,
         -0.1200, -0.1450],
        [-0.2406,  0.0555,  0.2076, -0.3265,  0.1260, -0.2892,  0.1068,  0.0882,
          0.2827,  0.1325, -0.1515, -0.0595,  0.2432,  0.0954, -0.0152, -0.2474,
          0.1093,  0.3327,  0.2901,  0.1741,  0.0231, -0.0845, -0.1553,  0.0597,
          0.1565,  0.0622,  0.0733, -0.0768,  0.0981, -0.0422, -0.0346, -0.1281,
         -0.0974,  0.0390,  0.0477,  0.1250,  0.1002, -0.0448, -0.2588, -0.0555,
         -0.1266,  0.2614,  0.1439, -0.0517, -0.0922, -0.0893, -0.1155,  0.0933,
          0.0698, -0.0533]])

Why = torch.tensor([[-0.2607, -0.1797],
        [ 0.0535,  0.1667],
        [ 0.1623,  0.0452],
        [ 0.0647, -0.1489],
        [-0.0087,  0.2329],
        [-0.0254,  0.1683],
        [ 0.3314,  0.3180],
        [-0.0280, -0.0388],
        [ 0.0504,  0.1590],
        [-0.1692,  0.0512],
        [-0.0499, -0.2180],
        [ 0.0518, -0.1771],
        [-0.1985,  0.2781],
        [ 0.2442,  0.1106],
        [-0.0769,  0.0066],
        [ 0.0396, -0.1901],
        [ 0.2852,  0.1101],
        [ 0.1771,  0.2348],
        [-0.0427,  0.3998],
        [ 0.3944, -0.0118],
        [ 0.5140, -0.0575],
        [ 0.0127, -0.1184],
        [ 0.1727, -0.2099],
        [-0.3124, -0.1508],
        [-0.0695, -0.2087],
        [-0.2225,  0.0460],
        [-0.2878,  0.1746],
        [ 0.2209, -0.2320],
        [ 0.3645,  0.2839],
        [ 0.1750, -0.1137],
        [-0.2313, -0.1732],
        [-0.8391,  0.4567],
        [ 0.2105, -0.2209],
        [ 0.0217, -0.0279],
        [-0.4355,  0.3213],
        [-0.2436,  0.3613],
        [-0.1415, -0.2574],
        [ 0.4312, -0.1981],
        [-0.0812, -0.2951],
        [ 0.1988, -0.0357],
        [ 0.2711, -0.8091],
        [ 0.2437,  0.2863],
        [-0.2435,  0.1952],
        [ 0.2801, -0.2416],
        [ 0.2897, -0.2687],
        [-0.0341,  0.0760],
        [ 0.3755, -0.1640],
        [ 0.0397, -0.2510],
        [ 0.1288,  0.3307],
        [-0.0946,  0.1220]])

In [4]:
W1 = Wxh
W2 = Wyh
W3 = Why

H = torch.tensor([[1., 0.]],dtype = torch.float32)


In [5]:
linear = (W2@W3).T
torch.linalg.eig(linear)

torch.return_types.linalg_eig(
eigenvalues=tensor([0.8841+0.2982j, 0.8841-0.2982j]),
eigenvectors=tensor([[0.0723-0.3286j, 0.0723+0.3286j],
        [0.9417+0.0000j, 0.9417-0.0000j]]))

## The dynamical system and the error system

In [ ]:
# Parameters
l = 1
g = 9.8 
b = 0.9
m = 2.
h = 0.1 

In [7]:
def f_value(x):
    y = torch.zeros_like(x)
    y[:,0] = x[:,0] + h*x[:,1]
    y[:,1] = x[:,1] + h*((-g*torch.sin(x[:,0])-b/m*x[:,1])/l)
    return y

In [8]:
def e_value(x,e):
    x_next = f_value(x)
    y = torch.tanh( x_next@H.T@W1 + x@W2 + e@W2)@W3 - x_next
    return y

## Neural network model for Lyapunov function V

In [ ]:
torch.manual_seed(42)  

class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_input, n_hidden, bias=False)
        self.output = nn.Linear(n_hidden,n_output, bias=False)
        self.to(device) 
        
    def forward(self,x):
        # Apply the square activation function
        x = self.hidden(x).pow(2)
        x = self.output(x).pow(2)
        return x

In [ ]:
def CheckLyapunov(e,x, V, V_next, config, epsilon):    
    
    x_ball= Expression(0)
    e_ball= Expression(0)
    V_difference = Expression(0)
    
    for i in range(len(x)):
        x_ball += x[i]*x[i]
        e_ball += e[i]*e[i]

    V_difference = V_next - V  - gamma*x_ball + alpha3*e_ball
    x_bound = logical_and(x_ball_lb**2 <= x_ball, x_ball <= x_ball_ub**2)
    e_bound = logical_and(e_ball_lb**2 <= e_ball, e_ball <= e_ball_ub**2)
    ball_in_bound = logical_and(x_bound, e_bound)
    V_positive = logical_and(V - alpha1*e_ball >= 0 , alpha2*e_ball - V >= 0)   
    condition = logical_imply(ball_in_bound, V_positive)

    condition = logical_and(condition,
                           logical_imply(ball_in_bound, V_difference <= epsilon))
                           
    return CheckSatisfiability(logical_not(condition),config)

In [11]:
def AddCounterexamples(x,CE,N): 
    c = []
    nearby= []
    for i in range(CE.size()):
        c.append(CE[i].mid())
        lb = CE[i].lb()
        ub = CE[i].ub()
        nearby_ = np.random.uniform(lb,ub,N)
        nearby.append(nearby_)
    for i in range(N):
        n_pt = []
        for j in range(x.shape[1]):
            n_pt.append(nearby[j][i])             
        x = torch.cat((x, torch.tensor([n_pt])), 0)
    return x

## Parameters

In [ ]:
'''
For learning 
'''
r_e = 2.
r = 1.
N = 1000            # sample size
D_in = 2            # input dimension
H1 = 10              # hidden dimension
D_out = 1           # output dimension

x = torch.Tensor(N, D_in).uniform_(-r, r)
e = torch.Tensor(N, D_in).uniform_(-r, r)              
e_0 = torch.zeros([1, 2])
e_0 = e_0.to(device)

'''
For verifying 
'''
e1 = Variable("e1")
e2 = Variable("e2")
# x1 = 2*e1
# x2 = 2*e2
x1 = Variable("x1")
x2 = Variable("x2")
x_ = [x1,x2]
e_ = [e1,e2]
# vars_ = [x1,x2]
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-3
config.number_of_jobs = 6
beta = 0.
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
x_ball_lb = 0.4
x_ball_ub = r
e_ball_lb = 0.4
e_ball_ub = r_e

# for verification of ISS Lyapunov conditions
alpha1 = 1e-2
alpha2 = 100.
alpha3 = alpha1
gamma = 100.

In [13]:
f_ex = [x1 + x2*h, x2 + h*((-g*sin(x1)-b/m*x2)/l)]

hidden_part = np.dot(f_ex, np.dot(H.T,W1)) + np.dot(x_, W2) + np.dot(e_, W2) 

hidden = []
for j in range(len(hidden_part)):
    hidden.append(tanh(hidden_part[j]))

xn_hat = np.dot(hidden, W3)

e_next_ex = []
for i in range(len(xn_hat)):
    e_next_ex.append(xn_hat[i] - f_ex[i])


## Learning and Falsification

In [ ]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    model = Net(D_in,H1, D_out)
    L = []
    i = 0 
    t = 0
    max_iters = 5000 # increase number of epoches if cannot find a valid LF
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        x = x.float()
        e = e.float()
        x = x.to(device)
        e = e.to(device)
        V_candidate = model(e)
        
        # e0 = model(e_0)
        f = f_value(x)
        e_next = e_value(x,e)
        V_next = model(e_next)
        V_difference = V_next - V_candidate + alpha3*e**2 - gamma*x**2
        # With tuning
        Lyapunov_risk = torch.sum(10.*F.relu(-V_candidate) + 5.*F.relu(-V_candidate + alpha1*x**2) + F.relu(V_candidate - alpha2*x**2) + F.relu(V_difference)) 

        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        # save the weights and biases 
        V_w1 = model.hidden.weight.data.cpu().numpy()
        V_w2 = model.hidden.weight.data.cpu().numpy()

        i += 1
        # Falsification with SMT solver
        if i % 50 == 0:
            
            # Candidate V
            z1 = np.dot(e_,V_w1.T)

            a1 = []
            for j in range(len(z1)):
                a1.append(z1[j]**2)
            z2 = np.dot(a1,V_w2)
            V_current = z2.item(0)**2

            # V Next
            z1_n = np.dot(e_next_ex,V_w1.T)

            a1_n = []
            for j in range(len(z1)):
                a1_n.append(z1_n[j]**2)
            z2_n = np.dot(a1_n,V_w2)
            V_next_ex = z2_n.item(0)**2

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            # beta = -np.maximum(beta, -0.02) # in case beta is too negative and cannot return any results
            result= CheckLyapunov(e_,x_, V_current, V_next_ex, config, beta) # SMT solver
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = x.to('cpu')
                e = e.to('cpu')
                x = AddCounterexamples(x,result,50)
                e = AddCounterexamples(e,result,50)
            else:  
                valid = True
                print("Satisfy conditions with beta = ", beta)
                print(V_current, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        

    stop = timeit.default_timer()

    # np.savetxt("w1_dp.txt", model.layer1.weight.data.cpu(), fmt="%s")
    # np.savetxt("w2_dp.txt", model.layer2.weight.data.cpu(), fmt="%s")
    # np.savetxt("b1_dp.txt", model.layer1.bias.data.cpu(), fmt="%s")
    # np.savetxt("b2_dp.txt", model.layer2.bias.data.cpu(), fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

0 Lyapunov Risk= 21.087953567504883
1 Lyapunov Risk= 20.869586944580078
2 Lyapunov Risk= 20.639053344726562
3 Lyapunov Risk= 20.389305114746094
4 Lyapunov Risk= 20.121841430664062
5 Lyapunov Risk= 19.836145401000977
6 Lyapunov Risk= 19.535789489746094
7 Lyapunov Risk= 19.215099334716797
8 Lyapunov Risk= 18.87702178955078
9 Lyapunov Risk= 18.524520874023438
10 Lyapunov Risk= 18.159461975097656
11 Lyapunov Risk= 17.77728843688965
12 Lyapunov Risk= 17.37855339050293
13 Lyapunov Risk= 16.95896339416504
14 Lyapunov Risk= 16.52936553955078
15 Lyapunov Risk= 16.09848976135254
16 Lyapunov Risk= 15.682760238647461
17 Lyapunov Risk= 15.282341957092285
18 Lyapunov Risk= 14.887857437133789
19 Lyapunov Risk= 14.509745597839355
20 Lyapunov Risk= 14.149782180786133
21 Lyapunov Risk= 13.802828788757324
22 Lyapunov Risk= 13.466343879699707
23 Lyapunov Risk= 13.137105941772461
24 Lyapunov Risk= 12.819686889648438
25 Lyapunov Risk= 12.52724552154541
26 Lyapunov Risk= 12.243270874023438
27 Lyapunov Risk= 

In [15]:
print(V_current)

pow(( - 0.463848 * pow(( - 0.463848 * e1 + 0.326237 * e2), 2) - 0.417419 * pow(( - 0.417419 * e1 + 0.254436 * e2), 2) - 0.226556 * pow(( - 0.226556 * e1 + 0.373596 * e2), 2) - 0.150608 * pow(( - 0.150608 * e1 + 0.0833274 * e2), 2) + 0.0529483 * pow((0.0529483 * e1 + 0.851326 * e2), 2) + 0.1475 * pow((0.1475 * e1 + 0.252181 * e2), 2) + 0.16729 * pow((0.16729 * e1 - 0.113263 * e2), 2) + 0.484694 * pow((0.484694 * e1 + 0.502514 * e2), 2) + 0.571265 * pow((0.571265 * e1 - 0.0627056 * e2), 2) + 0.656865 * pow((0.656865 * e1 - 0.603053 * e2), 2)), 2)


### Checking result with bounded beta if needed

In [16]:
beta = -0.02
start_ = timeit.default_timer() 
result= CheckLyapunov(e_,x_, V_current, V_next_ex, config, beta) # SMT solver
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with beta = ",beta)
    print(V_current, " is a Lyapunov function.")
t += (stop_ - start_)

Satisfy conditions with beta =  -0.02
pow(( - 0.463848 * pow(( - 0.463848 * e1 + 0.326237 * e2), 2) - 0.417419 * pow(( - 0.417419 * e1 + 0.254436 * e2), 2) - 0.226556 * pow(( - 0.226556 * e1 + 0.373596 * e2), 2) - 0.150608 * pow(( - 0.150608 * e1 + 0.0833274 * e2), 2) + 0.0529483 * pow((0.0529483 * e1 + 0.851326 * e2), 2) + 0.1475 * pow((0.1475 * e1 + 0.252181 * e2), 2) + 0.16729 * pow((0.16729 * e1 - 0.113263 * e2), 2) + 0.484694 * pow((0.484694 * e1 + 0.502514 * e2), 2) + 0.571265 * pow((0.571265 * e1 - 0.0627056 * e2), 2) + 0.656865 * pow((0.656865 * e1 - 0.603053 * e2), 2)), 2)  is a Lyapunov function.
